In [87]:
from datetime import datetime
import os
import re
import shutil
import math

import numpy as np
import pandas as pd
import wikipedia as wp

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_rows', 600)

In [13]:
def get_table(title, filename, match, use_cache=False):

    if use_cache and os.path.isfile(filename):
        pass
    else:
        html = wp.page(title).html()
        df = pd.read_html(html, header=0, match=match)[0]
        
        #df.to_csv(filename, header=True, index=False, encoding='utf-8')
            
    #df = pd.read_csv(filename)
    return df

In [14]:
title = 'List of S&P 500 companies'
filename = 'sp500.csv'
sp500 = get_table(title, filename, match='Symbol')

In [15]:
sp500.tail()

,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
500,YUM,Yum! Brands,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873
504,ZTS,Zoetis,reports,Health Care,Pharmaceuticals,"Parsippany, New Jersey",2013-06-21,1555280,1952


In [97]:
title = 'List of S&P 500 companies'
filename = 'changes_sp500.csv'
changes_sp500 = get_table(title, filename, match='Added')

In [77]:
changes_sp500.head()

,Date,Added,Added.1,Removed,Removed.1,Reason
0,Date,Ticker,Security,Ticker,Security,Reason
1,"March 2, 2022",MOH,Molina Healthcare,INFO,IHS Markit,S&P 500 constituent S&P Global Inc. acquired I...
2,"February 15, 2022",NDSN,Nordson,XLNX,Xilinx,S&P 500 constituent Advanced Micro Devices acq...
3,"February 3, 2022",NaN,NaN,GPS,Gap,Market capitalization change.[8]
4,"February 2, 2022",CEG,Constellation Energy,NaN,NaN,S&P 500 and 100 constituent Exelon Corp. spun ...


In [98]:
changes_sp500 = changes_sp500.drop(0)
changes_sp500['Date'] = pd.to_datetime(changes_sp500['Date'], format='%B %d, %Y')
changes_sp500.head()

,Date,Added,Added.1,Removed,Removed.1,Reason
1,2022-03-02,MOH,Molina Healthcare,INFO,IHS Markit,S&P 500 constituent S&P Global Inc. acquired I...
2,2022-02-15,NDSN,Nordson,XLNX,Xilinx,S&P 500 constituent Advanced Micro Devices acq...
3,2022-02-03,NaN,NaN,GPS,Gap,Market capitalization change.[8]
4,2022-02-02,CEG,Constellation Energy,NaN,NaN,S&P 500 and 100 constituent Exelon Corp. spun ...
5,2021-12-20,SBNY,Signature Bank,LEG,Leggett & Platt,Market capitalization change.[9]


In [79]:
changes_sp500.tail()

,Date,Added,Added.1,Removed,Removed.1,Reason
287,2000-12-05,SBL,Symbol Technologies,OI,Owens-Illinois,Market Cap changes.
288,2000-12-05,AYE,Allegheny Energy,GRA,WR Grace,Market Cap changes.
289,2000-12-05,ABK,Ambac Financial,CCK,Crown Holdings,Market Cap changes.
290,2000-07-27,JDSU,JDS Uniphase,RAD,RiteAid,Market Cap change.[225]
291,1999-12-07,YHOO,Yahoo!,LDW,Laidlaw,Market capitalization change.[226]


In [99]:
del changes_sp500['Added.1']
del changes_sp500['Removed.1']
del changes_sp500['Reason']

In [100]:
changes_sp500.tail()

,Date,Added,Removed
287,2000-12-05,SBL,OI
288,2000-12-05,AYE,GRA
289,2000-12-05,ABK,CCK
290,2000-07-27,JDSU,RAD
291,1999-12-07,YHOO,LDW


In [101]:
df = pd.DataFrame(columns=['DATE','REMOVED','ADDED'])

current_date = pd.to_datetime('2022-05-01', format='%Y-%m-%d')
adds = []
drops = []
for i in range(len(changes_sp500)) :
    if changes_sp500['Date'].values[i] == current_date :
        adds += [changes_sp500['Added'].values[i]]
        drops += [changes_sp500['Removed'].values[i]]
    else :
        df.loc[len(df)] = [current_date, adds, drops]

        current_date = changes_sp500['Date'].values[i]
        adds = [changes_sp500['Added'].values[i]]
        drops = [changes_sp500['Removed'].values[i]]

    if i == len(changes_sp500) - 1 :
        df.loc[len(df)] = [current_date, adds, drops]

C:\Users\Juan Pablo\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\dtypes\cast.py:881: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


In [102]:
df = df.drop(0)

In [104]:
df.head()

,DATE,REMOVED,ADDED
1,2022-03-02,[MOH],[INFO]
2,2022-02-15,[NDSN],[XLNX]
3,2022-02-03,[nan],[GPS]
4,2022-02-02,[CEG],[nan]
5,2021-12-20,"[SBNY, SEDG, FDS]","[LEG, HBI, WU]"


In [110]:
changes_sp500 = df.copy()

In [111]:
changes_sp500.head()

,DATE,REMOVED,ADDED
1,2022-03-02,[MOH],[INFO]
2,2022-02-15,[NDSN],[XLNX]
3,2022-02-03,[nan],[GPS]
4,2022-02-02,[CEG],[nan]
5,2021-12-20,"[SBNY, SEDG, FDS]","[LEG, HBI, WU]"


In [159]:
sp500_historical = pd.DataFrame(columns=['DATE','CONSTITUTENTS'])

constitutents = set(sp500['Symbol'])
sp500_historical.loc[len(sp500_historical)] = [changes_sp500['DATE'].values[0], list(sorted(constitutents))]
for i in range(len(changes_sp500)-1) :
    date = changes_sp500['DATE'].values[i+1]   

    for ticker in changes_sp500['REMOVED'].values[i] :
        constitutents.discard(ticker)
    for ticker in changes_sp500['ADDED'].values[i] :
        constitutents.add(ticker)

    constitutents = {x for x in constitutents if x==x}
    
    sp500_historical.loc[len(sp500_historical)] = [date, list(sorted(constitutents))]

C:\Users\Juan Pablo\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\dtypes\cast.py:881: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  element = np.asarray(element)


In [160]:
sp500_historical.tail()

,DATE,CONSTITUTENTS
221,2005-07-01,"[A, AA, AAPL, ABC, ABK, ABS, ABT, ACAS, ACE, A..."
222,2003-09-25,"[A, AA, AAPL, ABC, ABK, ABS, ABT, ACAS, ACE, A..."
223,2000-12-05,"[A, AA, AAPL, ABC, ABK, ABS, ABT, ACAS, ACE, A..."
224,2000-07-27,"[A, AA, AAPL, ABC, ABS, ABT, ACAS, ACE, ADBE, ..."
225,1999-12-07,"[A, AA, AAPL, ABC, ABS, ABT, ACAS, ACE, ADBE, ..."


In [161]:
sp500_historical.sort_values(by=['DATE'], ignore_index=True, inplace=True)

In [163]:
sp500_historical.head()

,DATE,CONSTITUTENTS
0,1999-12-07,"[A, AA, AAPL, ABC, ABS, ABT, ACAS, ACE, ADBE, ..."
1,2000-07-27,"[A, AA, AAPL, ABC, ABS, ABT, ACAS, ACE, ADBE, ..."
2,2000-12-05,"[A, AA, AAPL, ABC, ABK, ABS, ABT, ACAS, ACE, A..."
3,2003-09-25,"[A, AA, AAPL, ABC, ABK, ABS, ABT, ACAS, ACE, A..."
4,2005-07-01,"[A, AA, AAPL, ABC, ABK, ABS, ABT, ACAS, ACE, A..."


In [164]:
sp500_historical.to_csv('SP500_historical.csv')